<a href="https://colab.research.google.com/github/avionerman/computational_and_statistical/blob/main/svm_part_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports, Data Loading & Data subsets

In [1]:
# import numpy as np
# from tensorflow.keras.datasets import cifar10
# from sklearn.model_selection import train_test_split, GridSearchCV
!pip install cupy-cuda12x


In [2]:
!pip install tensorflow
import numpy as np
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split, GridSearchCV

start_bold = "\u001b[1m"
end_bold = "\033[0m"

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# x_train.shape, y_train.shape, x_test.shape, y_test.shape

class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

# # 10% train subset
x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.95, stratify=y_train, random_state=42)

# # 10% test subset
# x_test, _, y_test, _ = train_test_split(x_test, y_test, test_size=0.90, stratify=y_test, random_state=42)

# Preprocessing phase

## Flatten enablement

In [3]:
##### Flatten step #####

print(start_bold + "Flattening explanation:" + end_bold)
print("Flattening the data, is highly needed since both PCA and SVM cannot accept 3D data. \n"
"PCA calculates the covariance matrix among features. \n"
"That said, it needs an array of [samples x features], where each faeture is a column.\n")

x_train = x_train.reshape(len(x_train), -1)
x_test  = x_test.reshape(len(x_test), -1)
# x_train.shape, x_test.shape

Flattening explanation:
Flattening the data, is highly needed since both PCA and SVM cannot accept 3D data. 
PCA calculates the covariance matrix among features. 
That said, it needs an array of [samples x features], where each faeture is a column.



## Normalization enablement

In [4]:
##### Normalization step #####

print(start_bold + "Normalization explanation:" + end_bold)
print("I want to normalize my data mainly because I want to: \n"
"[1] to prevent my upcoming models from being dominated by large features.\n"
"[2] to feed a better scale for calculating distances for my models.\n"
"[3] and to help my PCA step with meaningful directions instead of large ones.\n")

x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0
# print(x_train.min(), x_train.max(), x_test.min(), x_test.max())

Normalization explanation:
I want to normalize my data mainly because I want to: 
[1] to prevent my upcoming models from being dominated by large features.
[2] to feed a better scale for calculating distances for my models.
[3] and to help my PCA step with meaningful directions instead of large ones.



## Standarization enablement

In [5]:
##### Standarization step #####

print(start_bold + "Standarization explanation:" + end_bold)
print("I want to standarize my data mainly because I want to: \n"
"[1] make my PCA work better since features with high deviation will not dominate.\n"
"[2] help my SVM to use all the features in a common scale.\n"
"*will use fit only for the training set, to prevent data leakage.\n"
"**fit learns info from the data, while transform applies the learned info to new data .\n"
"***mean should be 0, and std. dev should be 1.\n")

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

print("The train mean (μ) is:", x_train_scaled.mean(),
      "and the std. dev (σ) is:", x_train_scaled.std())

Standarization explanation:
I want to standarize my data mainly because I want to: 
[1] make my PCA work better since features with high deviation will not dominate.
[2] help my SVM to use all the features in a common scale.
*will use fit only for the training set, to prevent data leakage.
**fit learns info from the data, while transform applies the learned info to new data .
***mean should be 0, and std. dev should be 1.

The train mean (μ) is: -2.2331874e-09 and the std. dev (σ) is: 1.0000001


## PCA enablement

In [6]:
##### PCA (Principal component analysis) step #####

print(start_bold + "PCA explanation:" + end_bold)
print("To be updated: \n"
"*full: accurate, slow, memory heavy. \n"
"**auto: data shape (n_samples, n_features) -- recommended. \n")

from sklearn.decomposition import PCA
import time

start_time = time.time()
pca = PCA(n_components=0.90, svd_solver="auto", random_state=42)
x_train_pca = pca.fit_transform(x_train_scaled)
x_test_pca = pca.transform(x_test_scaled)
end_time = time.time()

print(f">>> The total PCA time was: {(end_time - start_time):.2f} seconds ({(end_time - start_time)/60:.2f} minutes)")
print(">>>",[float((pca.explained_variance_ratio_.sum())), (x_train_scaled.shape[1]), (x_train_pca.shape[1])] )

print("\n" + start_bold + "PCA results:" + end_bold)
print("Apparently, only 103 components are needed to explain almost 90% of the variance.\n"
"That looks good, because we has a huge dimensionality reduction from 3072 to 103, and at the same time"
"we didn't loose more than 10% of the total components.")

y_train_flat = y_train.ravel()
y_test_flat  = y_test.ravel()

PCA explanation:
To be updated: 
*full: accurate, slow, memory heavy. 
**auto: data shape (n_samples, n_features) -- recommended. 

>>> The total PCA time was: 21.10 seconds (0.35 minutes)
>>> [0.9005500674247742, 3072, 94]

PCA results:
Apparently, only 103 components are needed to explain almost 90% of the variance.
That looks good, because we has a huge dimensionality reduction from 3072 to 103, and at the same timewe didn't loose more than 10% of the total components.


# Test phase

The scope of this phase is only about checking if the pipeline works, if I get a logical accuracy and if the PCA works as expected on the dataset.

The optimization actions will come once the current step produces a logical baseline so we can start building on top of it in the following phase.

## Train an SVM with LinearSVC (once)

In [7]:
print(1)
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score
# import time
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     "C": [1]
# }

# grid = GridSearchCV(
#     LinearSVC(max_iter=5000),
#     param_grid=param_grid,
#     n_jobs=1,
#     verbose=3
# )

# start_time = time.time()
# grid.fit(x_train_pca, y_train_flat)
# train_time = time.time() - start_time

# print(f">>> The total LinearSVC time was: {train_time:.2f} seconds")

# y_train_pred_linearsvc = grid.predict(x_train_pca)
# y_test_pred_linearsvc  = grid.predict(x_test_pca)

# train_acc_w_linearsvc = accuracy_score(y_train_flat, y_train_pred_linearsvc)
# test_acc_w_linearsvc  = accuracy_score(y_test_flat, y_test_pred_linearsvc)

# print(f">>> The train accuracy was {train_acc_w_linearsvc:.4f} and the test accuracy was {test_acc_w_linearsvc:.4f}")

# print("\n" + start_bold + "LinearSVC results:" + end_bold)
# print("The train accuracy was close to 40%, same as the test accuracy.\n"
# "That looks good, firstly because we don't have to worry about overfitting, and secondly the pipeline is working as expected.\n")

1


## Train an SVM with SVC(kernel=linear) (once)

In [8]:
print(1)
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# import time

# model = SVC(kernel="linear", C=1.0, verbose=True)

# start_time = time.time()
# model.fit(x_train_pca, y_train_flat)
# train_time = time.time() - start_time

# print(f">>> The total SVC (linear) time was: {train_time:.2f} seconds")

# y_train_pred = model.predict(x_train_pca)
# y_test_pred  = model.predict(x_test_pca)

# train_acc = accuracy_score(y_train_flat, y_train_pred)
# test_acc  = accuracy_score(y_test_flat, y_test_pred)

# print(f">>> The train accuracy was {train_acc:.2f} and the test accuracy was {test_acc:.2f}")

1


# Model Selection - LinearSVC

In [9]:
import time
import pandas as pd
import cupy as cp

from cuml.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# transform data to GPU
x_train_gpu = cp.asarray(x_train_pca)
x_test_gpu  = cp.asarray(x_test_pca)
y_train_gpu = cp.asarray(y_train_flat)

param_grid = {
    "C": [0.1, 1, 10]
}

grid = GridSearchCV(
    estimator=LinearSVC(max_iter=5000, tol=1e-3),
    param_grid=param_grid,
    cv=5,
    verbose=2,
    n_jobs=-1
)

start_time = time.time()
grid.fit(x_train_gpu, y_train_flat)
grid_time = time.time() - start_time

print(f"\n >>> LinearSVC(GPU) total time: {grid_time:.2f} seconds")

best_model = grid.best_estimator_
best_params = grid.best_params_

print("\n >>> LinearSVC(GPU) Summary")
print("--------------------------")
print("Best mean CV accuracy:", grid.best_score_)
print("Best params:", best_params)

# make the predictions on GPU
y_train_pred = cp.asnumpy(best_model.predict(x_train_gpu))
y_test_pred  = cp.asnumpy(best_model.predict(x_test_gpu))

train_acc = accuracy_score(y_train_flat, y_train_pred)
test_acc  = accuracy_score(y_test_flat,  y_test_pred)

print("\nFinal Evaluation:")
print("Train acc:", train_acc)
print("Test acc:", test_acc)

results = pd.DataFrame(grid.cv_results_)
cv_table = results[["param_C", "mean_test_score", "std_test_score", "rank_test_score"]]

print("\n >>> Cross-Validation outcome table")
print(cv_table.sort_values("rank_test_score"))


Fitting 5 folds for each of 3 candidates, totalling 15 fits

 >>> LinearSVC(GPU) total time: 16.66 seconds

 >>> LinearSVC(GPU) Summary
--------------------------
Best mean CV accuracy: 0.34040000000000004
Best params: {'C': 0.1}

Final Evaluation:
Train acc: 0.4612
Test acc: 0.3461

 >>> Cross-Validation outcome table
   param_C  mean_test_score  std_test_score  rank_test_score
0      0.1           0.3404        0.021593                1
2     10.0           0.3404        0.022571                1
1      1.0           0.3400        0.022733                3


# Model Selection - SVC(kernel=rbf)

In [10]:
import time
import pandas as pd
import cupy as cp

from cuml.svm import SVC
from cuml.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# transform data to GPU
x_train_gpu = cp.asarray(x_train_pca)
x_test_gpu  = cp.asarray(x_test_pca)
y_train_gpu = cp.asarray(y_train_flat)

param_grid_rbf = {
    "C": [1, 10],
    "gamma": ["scale"]
}

rbf_grid = GridSearchCV(
    estimator=SVC(kernel="rbf"),
    param_grid=param_grid_rbf,
    cv=5,
    verbose=2,
    n_jobs=-1
)

start_time = time.time()
rbf_grid.fit(x_train_gpu, y_train_flat)
rbf_time = time.time() - start_time

print(f"\n >>> SVM(rbf, GPU) with GridSearchCV total execution time: {rbf_time:.2f} seconds")

best_rbf_score  = float(rbf_grid.best_score_)
best_rbf_params = rbf_grid.best_params_
best_rbf_model  = rbf_grid.best_estimator_

print("\n >>> SVM(rbf, GPU) Execution Summary")
print("----------------------------")
print(f"Best mean CV accuracy: {best_rbf_score:.4f}")
print(f"Best hyperparameters: {best_rbf_params}")
print(f"Best model: {best_rbf_model}")

# --- make the predictions on GPU, and transform back to CPU for metrics (acc)
y_train_pred_gpu = best_rbf_model.predict(x_train_gpu)
y_test_pred_gpu  = best_rbf_model.predict(x_test_gpu)

y_train_pred_rbf = cp.asnumpy(y_train_pred_gpu)
y_test_pred_rbf  = cp.asnumpy(y_test_pred_gpu)

train_acc_rbf = accuracy_score(y_train_flat, y_train_pred_rbf)
test_acc_rbf  = accuracy_score(y_test_flat,  y_test_pred_rbf)

print("\n >>> Final Evaluation of the best SVM(rbf, GPU)")
print(f"Train accuracy: {train_acc_rbf:.4f}")
print(f"Test accuracy:  {test_acc_rbf:.4f}")

rbf_results = pd.DataFrame(rbf_grid.cv_results_)

rbf_cv_table = rbf_results[
    ["param_C", "param_gamma", "mean_test_score", "std_test_score", "rank_test_score"]
]
print("\n >>> RBF Cross-Validation outcome table")
print(rbf_cv_table.sort_values("rank_test_score"))


Fitting 5 folds for each of 2 candidates, totalling 10 fits

 >>> SVM(rbf, GPU) with GridSearchCV total execution time: 12.25 seconds

 >>> SVM(rbf, GPU) Execution Summary
----------------------------
Best mean CV accuracy: 0.4132
Best hyperparameters: {'C': 1, 'gamma': 'scale'}
Best model: SVC()

 >>> Final Evaluation of the best SVM(rbf, GPU)
Train accuracy: 0.6984
Test accuracy:  0.4076

 >>> RBF Cross-Validation outcome table
   param_C param_gamma  mean_test_score  std_test_score  rank_test_score
0        1       scale           0.4132        0.019904                1
1       10       scale           0.4104        0.017727                2


# Model Selection - SVC(kernel=linear)

In [ ]:
import time
import pandas as pd
import cupy as cp

from cuml.svm import SVC
from cuml.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# transform to GPU
x_train_gpu = cp.asarray(x_train_pca)
x_test_gpu  = cp.asarray(x_test_pca)

param_grid_linear = {
    "C": [0.1, 1, 10]
}

linear_grid = GridSearchCV(
    estimator=SVC(kernel="linear"),
    param_grid=param_grid_linear,
    cv=5,
    verbose=2,
    n_jobs=-1
)

start_time = time.time()
linear_grid.fit(x_train_gpu, y_train_flat)
linear_time = time.time() - start_time

print(f"\n >>> SVM(linear, GPU) with GridSearchCV total execution time: {linear_time:.2f} seconds")

best_linear_score  = float(linear_grid.best_score_)
best_linear_params = linear_grid.best_params_
best_linear_model  = linear_grid.best_estimator_

print("\n >>> SVM(linear, GPU) Execution Summary")
print("----------------------------")
print(f"Best mean CV accuracy: {best_linear_score:.4f}")
print(f"Best hyperparameters: {best_linear_params}")
print(f"Best model: {best_linear_model}")

# make predictions on GPU, and transform results back to CPU
y_train_pred_gpu = best_linear_model.predict(x_train_gpu)
y_test_pred_gpu  = best_linear_model.predict(x_test_gpu)

y_train_pred_linear = cp.asnumpy(y_train_pred_gpu)
y_test_pred_linear  = cp.asnumpy(y_test_pred_gpu)

train_acc_linear = accuracy_score(y_train_flat, y_train_pred_linear)
test_acc_linear  = accuracy_score(y_test_flat,  y_test_pred_linear)

print("\n >>> Final Evaluation of the best SVM(linear, GPU)")
print(f"Train accuracy: {train_acc_linear:.4f}")
print(f"Test accuracy:  {test_acc_linear:.4f}")

linear_results = pd.DataFrame(linear_grid.cv_results_)

linear_cv_table = linear_results[
    ["param_C", "mean_test_score", "std_test_score", "rank_test_score"]
]
print("\n >>> Linear Cross-Validation outcome table")
print(linear_cv_table.sort_values("rank_test_score"))


[2025-12-07 17:47:00.279] [CUML] [warning] SVC with the linear kernel can be much faster using the specialized solver provided by LinearSVC. Consider switching to LinearSVC if tranining takes too long.
Fitting 5 folds for each of 3 candidates, totalling 15 fits


# Model Selection - SVC(kernel=poly)



In [ ]:
param_grid_poly = {
    "C": [1, 10],
    "degree": [2, 3]
}

grid_poly = GridSearchCV(
    estimator=SVC(kernel="poly", gamma="scale", coef0=0),
    param_grid=param_grid_poly,
    cv=5,
    verbose=2,
    n_jobs=-1
)

start = time.time()
grid_poly.fit(x_train_pca, y_train_flat)
time_poly = time.time() - start

print(f"\n>>> SVC(poly) GridSearchCV time: {time_poly:.2f} sec")
print("Best params:", grid_poly.best_params_)
print("Best CV score:", grid_poly.best_score_)

best_poly = grid_poly.best_estimator_

y_train_pred_poly = best_poly.predict(x_train_pca)
y_test_pred_poly  = best_poly.predict(x_test_pca)

train_acc_poly = accuracy_score(y_train_flat, y_train_pred_poly)
test_acc_poly  = accuracy_score(y_test_flat,  y_test_pred_poly)

print("\n>>> Final SVC (poly)")
print(f"Train accuracy: {train_acc_poly:.4f}")
print(f"Test accuracy:  {test_acc_poly:.4f}")


# kNN & NCC models

In [ ]:
import time
import numpy as np
import cupy as cp

from cuml.neighbors import KNeighborsClassifier as cuKNN  # GPU KNN
from sklearn.neighbors import NearestCentroid             # CPU NCC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

from scipy import sparse

# transform data for GPU
if sparse.issparse(x_train_pca):
    x_train_dense = x_train_pca.toarray()
    x_test_dense  = x_test_pca.toarray()
else:
    x_train_dense = x_train_pca
    x_test_dense  = x_test_pca

x_train_gpu = cp.asarray(x_train_dense)
x_test_gpu  = cp.asarray(x_test_dense)
y_train_gpu = cp.asarray(y_train_flat)

k_values = list(range(1, 15))

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

best_k = None
best_cv_score = -1.0
k_scores = []

start = time.time()

print("\n>>> Starting KNN (GPU) cross-validation over k...\n")
for k in k_values:
    fold_scores = []

    for train_idx, val_idx in cv.split(x_train_pca, y_train_flat):


        x_tr_gpu = x_train_gpu[train_idx]
        y_tr_gpu = y_train_gpu[train_idx]
        x_val_gpu = x_train_gpu[val_idx]
        y_val = y_train_flat[val_idx]

        knn_gpu = cuKNN(n_neighbors=k)
        knn_gpu.fit(x_tr_gpu, y_tr_gpu)

        # predict with GPU and then transform to CPU
        y_val_pred_gpu = knn_gpu.predict(x_val_gpu)
        y_val_pred = cp.asnumpy(y_val_pred_gpu)

        fold_scores.append(accuracy_score(y_val, y_val_pred))

    mean_score = float(np.mean(fold_scores))
    k_scores.append((k, mean_score))
    print(f"k={k:3d} | mean CV accuracy={mean_score:.4f}")

    if mean_score > best_cv_score:
        best_cv_score = mean_score
        best_k = k

knn_cv_time = time.time() - start

print(f"\n>>> Best k from CV: {best_k} with mean accuracy {best_cv_score:.4f}")
print(f">>> KNN CV search time (GPU): {knn_cv_time:.2f} sec\n")

# train the final kNN using the best k on full training set (with GPU)
best_knn_gpu = cuKNN(n_neighbors=best_k)

start = time.time()
best_knn_gpu.fit(x_train_gpu, y_train_gpu)
knn_train_time = time.time() - start

start = time.time()
y_test_pred_gpu = best_knn_gpu.predict(X_test_gpu)
knn_test_time = time.time() - start

# bring the predictions for accuracy and transform back to CPU
y_test_pred = cp.asnumpy(y_test_pred_gpu)
y_train_pred = cp.asnumpy(best_knn_gpu.predict(x_train_gpu))

knn_train_acc = accuracy_score(y_train_flat, y_train_pred)
knn_test_acc  = accuracy_score(y_test_flat,  y_test_pred)

print("\n KNN (k-Nearest Neighbors, GPU)\n")
print(f"Best k from CV: ", best_k)
print(f"CV search time (GPU): {knn_cv_time:.2f} sec")
print(f"Train time (best k only): {knn_train_time:.2f} sec")
print(f"Test time: {knn_test_time:.2f} sec\n")
print(f"Train accuracy (best k): {knn_train_acc:.4f}")
print(f"Test accuracy  (best k): {knn_test_acc:.4f}\n")



ncc = NearestCentroid()

start = time.time()
ncc.fit(x_train_pca, y_train_flat)
ncc_train_time = time.time() - start

start = time.time()
y_test_pred_ncc = ncc.predict(x_test_pca)
ncc_test_time = time.time() - start

ncc_train_acc = ncc.score(x_train_pca, y_train_flat)
ncc_test_acc  = ncc.score(x_test_pca,  y_test_flat)

print("\n NCC (Nearest Class Centroid)\n")
print(f"Train time: {ncc_train_time:.4f} sec")
print(f"Test time: {ncc_test_time:.4f} sec\n")
print(f"Train accuracy: {ncc_train_acc:.4f}")
print(f"Test accuracy: {ncc_test_acc:.4f}\n")


# Summary results

In [ ]:
import pandas as pd

summary = pd.DataFrame([
    ["LinearSVC",  train_acc, test_acc, grid_time],
    ["RBF SVM",    train_acc_rbf,    test_acc_rbf,    rbf_time],
    ["Linear SVM", train_acc_linear, test_acc_linear, linear_time],
    ["Poly SVM",   train_acc_poly,   test_acc_poly,   time_poly],
    ["kNN",  knn_train_acc,    knn_test_acc,    knn_train_time],
    ["NCC",        ncc_train_acc,    ncc_test_acc,    ncc_train_time],
], columns=["Model", "Train Accuracy", "Test Accuracy", "Train Time (s)"])

print("\n Summary results")
print(summary)


# Cateforization examples (visualization)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def show_examples(images, y_true, y_pred, class_names, correct=True, n=6, title=""):
    """
    images: original x_test (not PCA)
    y_true: real labels
    y_pred: predicted labels
    correct=True -> show correct predictions
    correct=False -> show incorrect predictions
    """

    if correct:
        idx = np.where(y_true == y_pred)[0]
    else:
        idx = np.where(y_true != y_pred)[0]

    if len(idx) == 0:
        print("No examples found for this case.")
        return

    chosen = np.random.choice(idx, size=min(n, len(idx)), replace=False)

    plt.figure(figsize=(15, 4))
    plt.suptitle(title, fontsize=16)

    for i, j in enumerate(chosen):
        plt.subplot(1, n, i + 1)
        plt.imshow(images[j].reshape(32, 32, 3))
        plt.axis("off")
        plt.title(f"T: {class_names[y_true[j]]}\nP: {class_names[y_pred[j]]}")

    plt.show()


show_examples(
    x_test,
    y_test_flat,
    y_test_pred,
    class_names,
    correct=True,
    n=6,
    title="LinearSVC – Correct Predictions"
)

show_examples(
    x_test,
    y_test_flat,
    y_test_pred,
    class_names,
    correct=False,
    n=6,
    title="LinearSVC – Incorrect Predictions"
)



show_examples(
    x_test,
    y_test_flat,
    y_test_pred_rbf,
    class_names,
    correct=True,
    n=6,
    title="RBF SVM – Correct Predictions"
)

show_examples(
    x_test,
    y_test_flat,
    y_test_pred_rbf,
    class_names,
    correct=False,
    n=6,
    title="RBF SVM – Incorrect Predictions"
)



show_examples(
    x_test,
    y_test_flat,
    y_test_pred_linear,
    class_names,
    correct=True,
    n=6,
    title="Linear SVM – Correct Predictions"
)

show_examples(
    x_test,
    y_test_flat,
    y_test_pred_linear,
    class_names,
    correct=False,
    n=6,
    title="Linear SVM – Incorrect Predictions"
)




show_examples(
    x_test,
    y_test_flat,
    y_test_pred_poly,
    class_names,
    correct=True,
    n=6,
    title="Poly SVM – Correct Predictions"
)

show_examples(
    x_test,
    y_test_flat,
    y_test_pred_poly,
    class_names,
    correct=False,
    n=6,
    title="Poly SVM – Incorrect Predictions"
)
